# Action Provider Demo: send requests to an AP endpoint

## 1. Setup

### 1.1 Select an Action Provider host

In [ ]:
"""Use Diaspora AP directly through the access endpoint."""

from __future__ import annotations

import json
from random import choice
from string import ascii_uppercase
from time import time

import requests
from diaspora_event_sdk import Client as GlobusClient

PRINT_RESPONSE = False

ap_endpoint = 'http://127.0.0.1:8000/'
# ap_endpoint = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com'
print('AP endpoint selected:', ap_endpoint)

### 1.2 Retrieve credential

In [ ]:
# c = GlobusClient()
# c.logout()

In [ ]:
"""A hacky way to retrieve access token."""
c = GlobusClient()
print("User's OpenID:", c.subject_openid)
tokens = c.login_manager._token_storage.get_token_data(
    '2b9d2f5c-fa32-45b5-875b-b24cd343b917',
)
access_token = tokens['access_token']
print("User's access token:", access_token)

### 1.3 Select a topic

In [ ]:
topic = 'topic1-' + c.subject_openid[-12:]
print(c.register_topic(topic))
print(c.list_topics())
print('Topic to produce/consume:', topic)

### 1.4 Set request header

In [ ]:
headers = {
    'authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

### 1.5 Helper functions

In [ ]:
def request_and_get_response(
    request_url: str,
    request_header: dict,
    request_data: dict,
) -> None:
    """Send API request and get response."""
    response = requests.post(
        request_url,
        headers=request_header,
        json=request_data,
    )

    # print('Response status code:', response.status_code)
    try:
        response_content = response.content.decode('utf-8')
        parsed_content = json.loads(response_content)
        print(parsed_content)
        return parsed_content

    except Exception:
        print('Should not happen - exception raised')
        print(response.content)

In [ ]:
def random_request_id() -> str:
    """Get a random request ID."""
    return str(''.join(choice(ascii_uppercase) for i in range(12)))

## 2.0 Produce a single message to AP with `key` and `value`

### 2.0.1 Produce a single message without `key`

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'value': {'content': 'hello world1'},
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 2.0.2 Produce a single message to AP with `key`

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'key': 'my-key-123',
        'value': {'content': 'hello world1'},
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

## 2.1 Produce a batch of messages to AP with `keys` and `msgs`

### 2.1.1 Produce messages to AP without keys

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 2.1.2 Produce messages to AP with a key

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'argonne national laboratory'},
            {'content': 'university of chicago'},
            {'content': 'johns hopkins university'},
        ],
        'keys': 'my-key-123',
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 2.1.3 Produce messages to AP with a list of keys

In [ ]:
data = {
    'request_id': '231',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1', 'my-key2', 'my-key3'],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 2.1.2: Error case: `msgs` does not exist

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

### 2.1.3: Error Case: `msgs` is empty

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

### 2.1.4: Error case: the topic does not exist (takes 10 seconds)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': '__badtopic',
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

### 2.1.7 Error case: the length of `keys` does not match the length of `msgs`

In [ ]:
data = {
    'request_id': '232',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1', 'my-key2'],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

## 3.1 Consume messages without filters

### 3.1.0 Produce a few more messages

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 3.1.1: Consume all messages (without `ts`, without `group_id`)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.1.2: Consume messages within one hour (with `ts`, without `group_id`)

In [ ]:
one_hour_ago = (int(time()) - 3600) * 1000
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': one_hour_ago,
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.1.3: Error case: the topic does not exist

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': '__badtopic',
        'ts': 1717532033372,
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

### 3.1.4: Error case: the user does not have access

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': 'diaspora-cicd',
        'ts': 1715930522000,
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

## 3.2 Blocking consume returns `ACTIVE`, retry with the same `request_id`

### 3.2.1 Produce a few more messages

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 3.2.2: Consume messages within one hour (with `ts`, with `group_id`)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': one_hour_ago,
        'group_id': 'my-group12',
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.2.3 The same request body returns `ACTIVE` status with no message

In [ ]:
consume_request_id = random_request_id()
print(consume_request_id)

data = {
    'request_id': consume_request_id,
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': one_hour_ago,
        'group_id': 'my-group12',
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'ACTIVE'

### 3.2.4 Produce a few more messages

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 3.2.5 Retry the same consume request with `request_id`

In [ ]:
data = {
    'request_id': consume_request_id,
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': one_hour_ago,
        'group_id': 'my-group12',
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

## 3.3 Blocking consume returns `ACTIVE`, retry with the same `action_id`


### 3.3.1 Create an `ACTIVE` request
Execute code in Section 3.2 first to use up all recent messages

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': one_hour_ago,
        'group_id': 'my-group12',
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'ACTIVE'
action_id = resp['action_id']

### 3.3.2 Produce a few more messages

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 3.3.3 Retry the consume request with `action_id` on the `status` endpoint

In [ ]:
print(action_id)

response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['status'] == 'SUCCEEDED'

## 3.4 Consume with filters

### 3.4.1 Consume with a Prefix Filter (can also use `ts` or `group_id`)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.4.2 Consume with a filter that has multiple conditions (cond1 AND cond2)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {
                'Pattern': {
                    'value': {
                        'content': [
                            {'prefix': 'hello', 'suffix': 'world2'},
                        ],
                    },
                },
            },
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.4.3 Consume with multiple filters (filter1 OR filter2)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'hello world2'}]}}},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.4.4 Consume with multiple filters (repeated filters do not return repeated msgs)

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'hello world1'}]}}},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.4.5 Error case: consume with a bad filter

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {
                'BadPattern': {
                    'value': {'content': [{'prefix': 'hello world1'}]},
                },
            },
            {'Pattern': {'value': {'content': [{'suffix': 'hello world1'}]}}},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'FAILED'
print(resp['details']['error'])

## 3.5 Consume with `ts`, `group_id` and `filters`, check status, and release

### 3.5.1 Produce a few more messages

In [ ]:
ts_now = int(time()) * 1000

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'

### 3.5.2 Consume with `ts`, `group_id` and `filters`

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'group_id': 'my-group12',
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'SUCCEEDED'
action_id = resp['action_id']
for topic_partition, messages in resp['details'].items():
    print('topic =', topic_partition, 'number of msgs =', len(messages))

### 3.5.3 Check the request status through `status` endpoint

In [ ]:
print(action_id)

response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['status'] == 'SUCCEEDED'

### 3.5.4 Error case: Check an non-exist `action_id`

In [ ]:
bad_action_id = action_id[:15]

response = requests.get(
    f'{ap_endpoint}/{bad_action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['code'] == 'ActionNotFound'

### 3.5.5 Release the request through `release` endpoint

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/release',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['status'] == 'SUCCEEDED'

### 3.5.6 Release the requests again returns `ActionNotFound`

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/release',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['code'] == 'ActionNotFound'

## 3.6 Blocking with `ts`, `group_id` and `filters`, check status, cancel, and release

### 3.6.1 Create an `ACTIVE` request
Execute code in Section 3.5 first to use up all recent messages

In [ ]:
data = {
    'request_id': random_request_id(),
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': ts_now,
        'group_id': 'my-group12',
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
        ],
    },
}
resp = request_and_get_response(f'{ap_endpoint}/run', headers, data)
assert resp['status'] == 'ACTIVE'
action_id = resp['action_id']

### 3.6.2 Check the request status through `status` endpoint

In [ ]:
print(action_id)

response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)

### 3.6.3 Error case: Attempt to release an active request

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/release',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['code'] == 'ActionConflict'

### 3.6.4 Cancel the request status through `cancel` endpoint

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/cancel',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['status'] == 'FAILED'

### 3.6.5 Cancel the request again returns idempotent results

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/cancel',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['status'] == 'FAILED'

### 3.6.6 Release the request

In [ ]:
response = requests.post(
    f'{ap_endpoint}/{action_id}/release',
    headers={'authorization': f'Bearer {access_token}'},
)

response_content = response.content.decode('utf-8')
resp = json.loads(response_content)
print(resp)
assert resp['status'] == 'FAILED'